In [ ]:
from python.animal_shelter import AnimalShelter
#from flask import Flask
from jupyter_plotly_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
import uuid

###########################
# Data Manipulation / Model
###########################

# FIXED update with your username and password and CRUD Python module name
username = "aacuser"
password = "4321"
clientObj = AnimalShelter(username, password)

one = "one"
many = "many"
data = clientObj.read({},many)
identifier = 'UUID: ' + str(uuid.uuid1())

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(data, columns=['id','age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])

df['id'] = df['animal_type']='Dog'
df.set_index('id', inplace=True, drop=False)

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = dbc.Container([
    dbc.Row([
        dbc.NavItem([
            html.A([
                html.Img(src=app.get_asset_url("Grazioso_Salvare_Logo.PNG"), height="150px", id='img-in'),
                    ], href='http://www.snhu.edu')
            ]),
        dbc.NavItem([html.H4(children="Developed by: Mark W. Meyer, " + identifier, id='id-in',
                       className='text-center text-success, mb-1'),
                    ]),
        dbc.Col([
            html.Div([
                html.H1(children="Grazioso Salvare's Dashboard", id='title-in',
                       className='text-center text-primary, mb-4'),
                html.Hr(),
                html.Div(id='title-out'),     
            ]), 
        ], width=12),             
    ]),
    
    dbc.Row([
        html.Div(
        [
            dcc.Tabs(id="tabs", value='tab-wtr-graph', children=[
                dcc.Tab(label="Water Rescue", id="tab-wtr", value='tab-wtr-list'),
                dcc.Tab(label="Mountain and Wilderness", id="tab-mtn", value='tab-mtn-list'),
                dcc.Tab(label="Disaster and Individual Tracking", id="tab-dis", value='tab-dis-list'),
            ]),
            html.Div(id="tab-content-graph"),
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.Div([
                dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i, "deletable": True, "selectable": True} 
                     for i in df.columns
                    if i != 'id'
            ],
            data=df.to_dict('records'),
            
            style_cell=dict(textAlign='left'),
            style_header=dict(backgroundColor="paleturquoise"),
            style_data=dict(backgroundColor="lavender", whiteSpace="normal", height="auto"),
            editable=True,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            column_selectable="single",
            row_selectable="multi",
            row_deletable=True,
            selected_rows=[],
            hidden_columns=['id',
                            'datetime',
                            'monthyear',
                            'outcome_subtype'],
            page_action="native",
            page_current= 0,
            page_size=10                     
    ), html.Div(id='table-container')
            ])
        ], width=12),
    ], justify="around"),
    
        dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='my-dpdn', multi=False,
                        options=[{'label': x, 'value': x}
                                for x in sorted(df['animal_type'].unique())]),
            dcc.Graph(id='scatter-fig', figure={})
        ], width={'size':6}),
        dbc.Col([
            dcc.Dropdown(id='my-dpdn2', multi=False,
                        options=[{'label': x, 'value': x}
                                for x in sorted(df['animal_type'].unique())]),
            dcc.Graph(id='my-map', figure={})
        ], width={'size':6}),
    ], justify="around"),
    
], fluid=True)


@app.callback(Output('tab-content-graph', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-wtr-list':
        return html.Div([
            html.H6("Labrador Retriever Mix"),
            html.H6("Chesapeake Bay Retriever"),
            html.H6("Newfoundland"),
        ])
    elif tab == 'tab-mtn-list':
        return html.Div([
            html.H6("German Shepherd"),
            html.H6("Alaskan Malamute"),
            html.H6("Old English Sheepdog"),
            html.H6("Siberian Husky"),
            html.H6("Rottweiler"),
        ])
    elif tab == 'tab-dis-list':
        return html.Div([
            html.H6("Doberman Pinscher"),
            html.H6("German Shepherd"),
            html.H6("Golden Retriever"),
            html.H6("Bloodhound"),
            html.H6("Rottweiler"),
        ])



@app.callback(
        Output('title-out', 'children'),
        [Input('title-in', 'children'),
         Input('id-in', 'children'),
        Input('img-in', 'children')]
)

@app.callback(
    Output('table-container', "children"),
    [Input('table', 'derived_virtual_row_ids'),
     Input('table', 'selected_row_ids'),
     Input('table', 'active_cell'),
    ]
)
def update_graphs(row_ids, selected_row_ids, active_cell):
    selected_id_set = set(selected_row_ids or [])

    if row_ids is None:
        dff = df
        row_ids = df['id']
    else:
        dff = df.loc[row_ids]

    active_row_id = active_cell['row_id'] if active_cell else None

    colors = ['#FF69B4' if id == active_row_id
              else '#7FDBFF' if id in selected_id_set
              else '#0074D9'
              for id in row_ids]

    return [
        dcc.Graph(
            id=column + '--row-ids',
            figure={
                'data': [
                    {
                        'x': dff['breed'],
                        'y': dff[column],
                        'type': 'bar',
                        'marker': {'color': colors},
                    }
                ],
                'layout': {
                    'xaxis': {'automargin': True},
                    'yaxis': {
                        'automargin': True,
                        'title': {'text': column}
                    },
                    'height': 250,
                    'margin': {'t': 10, 'l': 10, 'r': 10},
                },
            },
        )
                # check if column exists - user may have deleted it
                # If `column.deletable=False`, then you don't
                # need to do this check.
                for column in ["animal_id", "animal_type", "breed", "color",
                              "date_of_birth", "datetime", "monthyear", "name", "outcome_subtype",
                              "outcome_type", "sex_upon_outcome", "location_lat", "location_long",
                              "age_upon_outcome_in_weeks"
                              ] if column in dff
    ]


@app.callback(
        Output('scatter-fig', 'figure'),
        [Input('my-dpdn', 'value')]
)
def update_graph(animal_selected):
    dff = df[df['animal_type']==animal_selected]
    figln = px.scatter(dff, x='breed', y='outcome_type', color='age_upon_outcome_in_weeks')
    return figln


@app.callback(
    Output('my-map', 'figure'),
    [Input('my-dpdn2', 'children')]
)
def update_graph(animal_selected):
    dff = df[df['animal_type']==animal_selected]
    figplot = px.scatter_mapbox(dff, lat="location_lat", lon="location_long", color="monthyear",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10, mapbox_style="carto-positron")
    return figplot


if __name__=='__main__':
        app.run_server(port=8050)

Read successful
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /assets/Grazioso_Salvare_Logo.PNG HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Dec/2021 04:25:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/mark.meyer_snhu/.local/lib/python3.6/site-packages/dash/dash.py

127.0.0.1 - - [13/Dec/2021 04:25:45] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [13/Dec/2021 04:25:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:25:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:26:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:26:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:26:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:26:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:26:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Dec/2021 04:27:20] "POST /_dash-update-component HTTP/1.1" 200 -
